In [6]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv("/home/ubuntu/ARTrade/strategy/Demo/BTCUSDT_BINANCE_1m_2024_01_04.csv")

In [8]:
df = pd.DataFrame()

df["Open"] = data["open"]
df["High"] = data["high"]
df["Low"] = data["low"]
df["Close"] = data["close"]
df["Volume"] = data["volume"]
# df.index = data["open_time"]
# df.index = pd.to_datetime(df.index.astype(int) / 1000, unit='s')
df["open_time"] = data["open_time"]
df["open_time"] = pd.to_datetime(df["open_time"].astype(int) / 1000, unit='s')

print(df)

           Open     High      Low    Close   Volume           open_time
0       42314.0  42335.8  42289.6  42331.9  289.641 2024-01-01 00:00:00
1       42331.9  42353.1  42331.8  42350.4  202.444 2024-01-01 00:01:00
2       42350.4  42370.8  42349.6  42360.2  271.521 2024-01-01 00:02:00
3       42360.1  42405.8  42360.1  42405.8  392.238 2024-01-01 00:03:00
4       42405.7  42437.2  42405.7  42437.1  568.366 2024-01-01 00:04:00
...         ...      ...      ...      ...      ...                 ...
174230  60877.5  60890.2  60810.1  60834.3  130.191 2024-04-30 23:50:00
174231  60834.3  60835.3  60776.5  60791.9  119.424 2024-04-30 23:51:00
174232  60792.0  60822.0  60790.8  60790.8   66.904 2024-04-30 23:52:00
174233  60790.8  60800.0  60759.3  60759.3   77.052 2024-04-30 23:53:00
174234  60759.3  60776.0  60721.1  60731.6  134.499 2024-04-30 23:54:00

[174235 rows x 6 columns]


In [9]:
kdj = ta.kdj(df["High"], df["Low"], df["Close"], length=14)
df[["KDJ_K", "KDJ_D", "KDJ_J"]] = kdj

print(df)
df['trade_point'] = 0

# 金叉
golden_cross_condition = (df['KDJ_K'] > df['KDJ_D']) & (df['KDJ_K'].shift(1) <= df['KDJ_D'].shift(1)) & (df['KDJ_K'] <= 20)
df.loc[golden_cross_condition, 'trade_point'] = 1
# 死叉
death_cross_condition = (df['KDJ_K'] < df['KDJ_D']) & (df['KDJ_K'].shift(1) >= df['KDJ_D'].shift(1)) & (df['KDJ_K'] >= 80)
df.loc[death_cross_condition, 'trade_point'] = -1
print(df)


           Open     High      Low    Close   Volume           open_time  \
0       42314.0  42335.8  42289.6  42331.9  289.641 2024-01-01 00:00:00   
1       42331.9  42353.1  42331.8  42350.4  202.444 2024-01-01 00:01:00   
2       42350.4  42370.8  42349.6  42360.2  271.521 2024-01-01 00:02:00   
3       42360.1  42405.8  42360.1  42405.8  392.238 2024-01-01 00:03:00   
4       42405.7  42437.2  42405.7  42437.1  568.366 2024-01-01 00:04:00   
...         ...      ...      ...      ...      ...                 ...   
174230  60877.5  60890.2  60810.1  60834.3  130.191 2024-04-30 23:50:00   
174231  60834.3  60835.3  60776.5  60791.9  119.424 2024-04-30 23:51:00   
174232  60792.0  60822.0  60790.8  60790.8   66.904 2024-04-30 23:52:00   
174233  60790.8  60800.0  60759.3  60759.3   77.052 2024-04-30 23:53:00   
174234  60759.3  60776.0  60721.1  60731.6  134.499 2024-04-30 23:54:00   

            KDJ_K      KDJ_D      KDJ_J  
0             NaN        NaN        NaN  
1             N

In [10]:
rsi = ta.rsi(df['Close'], length=14)
macd = ta.macd(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
df['RSI'] = rsi
df[["MACDline", "MACDsignal", "MACDhist"]] = macd

In [11]:

df.to_csv("talib.csv", index=False)